In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install gym

In [3]:
!git clone https://github.com/FlemingDL/gym_fleming.git

!pip install -e gym_fleming

Cloning into 'gym_fleming'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 99 (delta 37), reused 86 (delta 24), pack-reused 0
Unpacking objects: 100% (99/99), done.
Obtaining file:///content/gym_fleming
  Running setup.py develop for gym-fleming


In [0]:
import sys
from contextlib import closing
from six import StringIO
from gym import utils
from gym.envs.toy_text import discrete
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import gym
import gym_fleming
from gym import wrappers

In [2]:

# env = gym.make('taxi_fleming-v0')
# env = gym.wrappers.Monitor(env, "./drive/My Drive/Colab Notebooks/RL Project/double_Q_5x5",  video_callable=lambda episode_id: True, force=True)
# env = gym.make('taxi_fleming-10x10-v0')
# env = gym.wrappers.Monitor(env, "./drive/My Drive/Colab Notebooks/RL Project/double_Q_10x10",  video_callable=lambda episode_id: True, force=True)
# env = gym.make('taxi_fleming-15x15-v0')
# env = gym.wrappers.Monitor(env, "./drive/My Drive/Colab Notebooks/RL Project/double_Q_15x15",  video_callable=lambda episode_id: True, force=True)
env = gym.make('taxi_fleming-20x20-v0')
# env = gym.wrappers.Monitor(env, "./drive/My Drive/Colab Notebooks/RL Project/double_Q_20x20",  video_callable=lambda episode_id: True, force=True)

#  env = gym.wrappers.Monitor(env, directory, video_callable=lambda episode_id: episode_id%10==0)
n_state = env.observation_space.n
print(n_state)

n_action = env.action_space.n
print(n_action)

8000
6


In [3]:
env.render()


+---------------------------------------+
|R: | : : : : | : : : : | : : : : | : :G|
| : | : : : : | : : : : | : : : : | : : |
| : : : : : : : : : : : : : : : : : : : |
| | : | : : | : | : : | : | : : | : | : |
| | : | : : | : | : : | : | : : | : | : |
| : | : : : : | : : : : | : : : : | : : |
| : | : : : : | : : : : | : : : : | : : |
| : : : : : : : : : : : : : : : : : : : |
| | : | : : | : | : : | : | : : | : | : |
| | : | : : | : | : : | : | : : | : | : |
| : | : : : : | : : : : | : : : : | : : |
| : | : : : : | : : : : | : : : : | : : |
| : : : : : : : : : : : : : : : : : : : |
| | : | : : | : | : : | : | : : | : | : |
| | : | : : | : | : : | : | : : | : | : |
| : | : : : : | : : : : | : : : : | : : |
| : | : : : : | : : : : | : : : : | : : |
| : : : : : : : : : : : : : : : : : : : |
| | : | : : | : | : : | : | : : | : | : |
|Y| : | : : | : | : : | : | : : | : |B: |
+---------------------------------------+



In [0]:

# class TaxiEnv(discrete.DiscreteEnv):
#     """
#     The Taxi Problem
#     from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
#     by Tom Dietterich
#     Description:
#     There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.
#     Observations: 
#     There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations. 
    
#     Passenger locations:
#     - 0: R(ed)
#     - 1: G(reen)
#     - 2: Y(ellow)
#     - 3: B(lue)
#     - 4: in taxi
    
#     Destinations:
#     - 0: R(ed)
#     - 1: G(reen)
#     - 2: Y(ellow)
#     - 3: B(lue)
        
#     Actions:
#     There are 6 discrete deterministic actions:
#     - 0: move south
#     - 1: move north
#     - 2: move east 
#     - 3: move west 
#     - 4: pickup passenger
#     - 5: dropoff passenger
    
#     Rewards: 
#     There is a reward of -1 for each action and an additional reward of +20 for delivering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.
    
#     Rendering:
#     - blue: passenger
#     - magenta: destination
#     - yellow: empty taxi
#     - green: full taxi
#     - other letters (R, G, Y and B): locations for passengers and destinations
    
#     state space is represented by:
#         (taxi_row, taxi_col, passenger_location, destination)
#     """
#     metadata = {'render.modes': ['human', 'ansi']}

In [0]:
>>> def double_q_learning(env, gamma, n_episode, alpha):
...     """
...     Obtain the optimal policy with off-policy double 
        Q-learning method
...     @param env: OpenAI Gym environment
...     @param gamma: discount factor
...     @param n_episode: number of episodes
...     @return: the optimal Q-function, and the optimal policy
...     """
...     n_action = env.action_space.n
...     n_state = env.observation_space.n
...     Q1 = torch.zeros(n_state, n_action)
...     Q2 = torch.zeros(n_state, n_action)
...     frames =[]
...     for episode in range(n_episode):
...         state = env.reset()
...         is_done = False
...         while not is_done:
# ...             env.render()
# ...             env.close()
...             action = epsilon_greedy_policy(state, Q1 + Q2)
...             next_state, reward, is_done, info = env.step(action)
...             if (torch.rand(1).item() < 0.5):
...                 best_next_action = torch.argmax(Q1[next_state])
...                 td_delta = reward + gamma * Q2[next_state][best_next_action]  - Q1[state][action]
...                 Q1[state][action] += alpha * td_delta
...             else:
...                 best_next_action = torch.argmax(Q2[next_state])
...                 td_delta = reward + gamma * Q1[next_state][best_next_action] - Q2[state][action]
...                 Q2[state][action] += alpha * td_delta
...             length_episode[episode] += 1
...             total_reward_episode[episode] += reward
# ...             frames.append({'frame': env.render(mode='ansi'),'state': state,'action': action,'reward': reward})
...             if is_done:
...                 break
...             state = next_state
# ...     env.close()
...     policy = {}
...     Q = Q1 + Q2


...     for state in range(n_state):
...         policy[state] = torch.argmax(Q[state]).item()
...     return Q, policy#,frames

In [0]:
>>> def gen_epsilon_greedy_policy(n_action, epsilon):
        def policy_function(state, Q):
            probs = torch.ones(n_action) * epsilon / n_action
            best_action = torch.argmax(Q[state]).item()
            probs[best_action] += 1.0 - epsilon
            action = torch.multinomial(probs, 1).item()
            return action
        return policy_function

In [0]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.2)

Gamma = 1, Alpha = 0.4, Epsilon = 0.1


141



In [0]:
>>> n_episode = 2000
>>> length_episode = [0] * n_episode
>>> total_reward_episode = [0] * n_episode

In [0]:
alpha_options = [0.4, 0.5, 0.6]
epsilon_options = [0.3, 0.1, 0.01]
gamma = 1

In [0]:
for alpha in alpha_options:
     for epsilon in epsilon_options:
         length_episode = [0] * n_episode
         total_reward_episode = [0] * n_episode
         epsilon_greedy_policy = gen_epsilon_greedy_policy(env.action_space.n, epsilon)
         double_q_learning(env, gamma, n_episode, alpha)
         reward_per_step = [reward/float(step) for 
                            reward, step in zip(
                        total_reward_episode, length_episode)]
         print('alpha: {}, epsilon: {}'.format(alpha, epsilon))
         print('Average reward over {} episodes: {}'.format(
            n_episode, sum(total_reward_episode) / n_episode))
         print('Average length over {} episodes: {}'.format(
            n_episode, sum(length_episode) / n_episode))
         print('Average reward per step over {} episodes: {}\n'.format(
            n_episode, sum(reward_per_step) / n_episode))

alpha: 0.4, epsilon: 0.3
Average reward over 2000 episodes: -3644.3325
Average length over 2000 episodes: 1596.939
Average reward per step over 2000 episodes: -2.1561892832753258

alpha: 0.4, epsilon: 0.1
Average reward over 2000 episodes: -3268.065
Average length over 2000 episodes: 1487.31
Average reward per step over 2000 episodes: -2.058981527068195

alpha: 0.4, epsilon: 0.01
Average reward over 2000 episodes: -3157.6005
Average length over 2000 episodes: 1450.2045
Average reward per step over 2000 episodes: -2.023086241049618

alpha: 0.5, epsilon: 0.3
Average reward over 2000 episodes: -3335.5225
Average length over 2000 episodes: 1401.8845
Average reward per step over 2000 episodes: -2.2272988285111603

alpha: 0.5, epsilon: 0.1
Average reward over 2000 episodes: -3081.2705
Average length over 2000 episodes: 1346.627
Average reward per step over 2000 episodes: -2.102831994340026

alpha: 0.5, epsilon: 0.01
Average reward over 2000 episodes: -2973.443
Average length over 2000 episod

In [0]:
#10x10
>>> gamma = 1
>>> alpha = 0.4
>>> epsilon = 0.01
>>> epsilon_greedy_policy = gen_epsilon_greedy_policy(env.action_space.n, epsilon)
>>> five_try = ['one','two','three','four','five']
# >>> five_try = ['one']
>>> for i in five_try:
>>>   length_episode = [0] * n_episode
>>>   total_reward_episode = [0] * n_episode
>>>   optimal_Q, optimal_policy = double_q_learning(env, gamma, n_episode, alpha)
>>>   vars()["length_episode_"+i] = length_episode
>>>   vars()["total_reward_episode_"+i] = total_reward_episode

In [0]:
import pandas as pd


length_df = pd.DataFrame({'20x20_one' : length_episode_one ,
                   '20x20_two' : length_episode_two,
                   '20x20_three' : length_episode_three,
                   '20x20_four' : length_episode_four,
                   '20x20_five' : length_episode_five
                   }
                    )

reward_df = pd.DataFrame({'20x20_one' : total_reward_episode_one ,
                   '20x20_two' : total_reward_episode_two,
                   '20x20_three' : total_reward_episode_three,
                   '20x20_four' : total_reward_episode_four,
                   '20x20_five' : total_reward_episode_five
                   }
                    )


In [9]:
length_df

,20x20_one,20x20_two,20x20_three,20x20_four,20x20_five
0,6440,3667,997,25508,13779
1,5425,4620,4916,16922,7782
2,11464,9534,3878,5873,1095
3,4884,4493,5958,4137,35037
4,5055,31396,20327,8500,2591
...,...,...,...,...,...
1995,53,678,80,426,600
1996,439,1038,89,880,603
1997,90,160,69,93,79
1998,190,749,405,409,438


In [10]:
reward_df

,20x20_one,20x20_two,20x20_three,20x20_four,20x20_five
0,-21161,-13312,-3946,-67085,-40173
1,-19741,-13491,-18755,-44792,-23727
2,-27427,-27054,-15620,-19091,-4242
3,-13575,-8792,-18006,-12333,-87576
4,-18930,-74863,-48773,-18559,-5522
...,...,...,...,...,...
1995,-50,-1323,-95,-891,-1335
1996,-1021,-2142,-149,-1885,-1320
1997,-96,-283,-75,-144,-130
1998,-304,-1628,-888,-910,-903


In [0]:
from google.colab import files
length_df.to_csv('20x20_0.4_0.01_length_result.csv') 
reward_df.to_csv('20x20_0.4_0.01_reward_result.csv') 
# files.download('filename.csv')

